In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import datasets
from torch.nn.utils.rnn import pad_sequence
from datasets import Dataset, DatasetDict
from tqdm.notebook import tqdm, trange
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import gc
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import gensim.downloader
# import string
import gensim
from string import punctuation
import torch
import torch.autograd as autograd
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
df = pd.read_excel('../df_clean_lemma.xlsx')
df.head()

,text,label,dataset,type,language,Google,multi_language,token,lemma,tweet_token,tweet_lemma
0,-מעשי הטרוריזם האחרונים של קיין... שביניהם פיצ...,-1,XED,sentence,he,-kane's recent acts of terrorism... including ...,0,kane s recent acts of terrorism including the ...,kane recent act terrorism including explosion ...,kane's recent acts of terrorism including the ...,kane's recent act terrorism including explosio...
1,- а помните как раньше с фруктов наклейки поо...,1,RuTweetCorp,tweet,ru,- and remember how we used to stick stickers o...,0,and remember how we used to stick stickers on ...,remember used stick sticker nail fruit busines...,and remember how we used to stick stickers on ...,remember used stick sticker nail fruit busines...
2,"- если обычная девушка - загадка, то я супер ...",1,RuTweetCorp,tweet,ru,"- if an ordinary girl is a mystery, then i am ...",0,if an ordinary girl is a mystery then i am a s...,ordinary girl mystery super secret state secret,if an ordinary girl is a mystery then i am a s...,ordinary girl mystery super secret state secre...
3,"- парни-это подобие унитазов: или заняты, или...",1,RuTweetCorp,tweet,ru,- guys are like toilets: either busy or shitty),0,guys are like toilets either busy or shitty,guy like toilet either busy shitty,guys are like toilets either busy or shitty,guy like toilet either busy shitty
4,"- я так то добрый , но на хуй вам придется вс...",1,RuTweetCorp,tweet,ru,"- i'm so kind, but fuck you you still have to ...",0,i m so kind but fuck you you still have to go,kind fuck still go,i'm so kind but fuck you you still have to go :-),kind fuck still go :-)


In [8]:
def data_preprocess(df, text_column, label_column, datatype_column, datatype, fillna, shuffle_flg):
    if fillna=='drop':
        df.dropna().reset_index(drop=True,inplace=False)
    else:
        if type(fillna[0]) == str:
            values = {text_column: fillna[0], label_column: fillna[1]}
            if use_datatype:
                values[datatype_column] = fillna[0]
        else:
            values = {text_column: fillna[1], label_column: fillna[0]}
            if datatype_column is not None:
                values[datatype_column] = fillna[1]            
        df.fillna(value=values)
    df = df[df[text_column].isna()==False]
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    if datatype_column is not None and datatype != 'all':
        df = df[df[datatype_column] == datatype]
        df.reset_index(drop=True, inplace=True)
    if shuffle_flg:
        df = shuffle(df)
    values = None, None; del values; gc.collect()
    return df


In [7]:
emb_name = 'glove.6B.300d.txt'
try:
    word2vec = gensim.downloader.load(emb_name)
except:
    try:
        word2vec = KeyedVectors.load_word2vec_format('./Эмбеддинги/'+emb_name, binary=False)
    except:
        word2vec = KeyedVectors.load_word2vec_format('./Эмбеддинги/'+emb_name, binary=True)


In [10]:
def encode(word):
    if word in word2idx.keys():
        return word2idx[word]
    return word2idx["unk"]
word2idx = {word: idx for idx, word in enumerate(word2vec.index_to_key )}


In [11]:
def make_emb(emb_vectors, sent, emb_type, min_length, max_length):
    ans = []
    flag = True
    word_count = 0
    if emb_type == 'mean':
        if type(sent)==str:
            sent = sent.split()
            for word in sent:
                try:
                    if flag:
                        ans = emb_vectors[word]
                        word_count += 1
                        flag = False
                    else:
                        ans = list(map(sum, zip(ans, emb_vectors[word])))
                        word_count += 1
                except:
                    continue
        elif type(sent)==list:
            sent = sent
            for word in sent:
                try:
                    if flag:
                        ans = emb_vectors[word]
                        word_count += 1
                        flag = False
                    else:
                        ans = list(map(sum, zip(ans, emb_vectors[word])))
                        word_count += 1
                except:
                    continue   
        if word_count > min_length and word_count < max_length:
            ans = list(np.array(ans) / word_count)
            sent = None; del sent; 
            return ans
        else:
            ans = list(np.array(ans) / word_count)
            ans, sent = None, None; del ans, sent; 
            return []        
    elif emb_type == 'sequence':
        if type(sent)==str:
            sent = sent.split()
            for word in sent:
                try:
                    if flag:
                        ans = emb_vectors[word]
                        word_count += 1
                        flag = False
                    else:
                        ans = ans.append(emb_vectors[word])
                        word_count += 1
                except:
                    continue
        elif type(sent)==list:
            sent = sent
            for word in sent:
                try:
                    if flag:
                        ans = [emb_vectors[word]]
                        word_count += 1
                        flag = False
                    else:
                        ans.append(emb_vectors[word])
                        word_count += 1
                except:
                    continue   
        if word_count > min_length and word_count < max_length:
            sent = None; del sent; 
            return ans
        else:
            ans, sent = None, None; del ans, sent; 
            return []   


In [9]:
df1 = data_preprocess(df, text_column='lemma', label_column='label', datatype_column='dataset', datatype='all', fillna='drop', shuffle_flg=True)
df1['label'] = [df1['label'][i]+1 for i in range(df1.shape[0])]
df1


,text,label,dataset,type,language,Google,multi_language,token,lemma,tweet_token,tweet_lemma
293969,sam harper was a marine who was killed by frie...,0,Large movie review dataset,review,en,sam harper was a marine who was killed by frie...,0,sam harper was a marine who was killed by frie...,sam harper marine killed friendly fire war kuw...,sam harper was a marine who was killed by frie...,sam harper marine killed friendly fire war kuw...
461247,отличный отель останавливались с женой во врем...,2,clothing e-comm,review,ru,"excellent hotel, my wife and i stayed during c...",0,excellent hotel my wife and i stayed during ch...,excellent hotel wife stayed check got free roo...,excellent hotel my wife and i stayed during ch...,excellent hotel wife stayed check-in got free ...
159219,excellent product hair striaght dry matter min...,2,Multi-Domain,review,en,excellent product hair striaght dry matter min...,0,excellent product hair striaght dry matter min...,excellent product hair striaght dry matter minute,excellent product hair striaght dry matter min...,excellent product hair striaght dry matter minute
121086,"almost no martial arts and what there was, app...",2,Large movie review dataset,review,en,"almost no martial arts and what there was, app...",0,almost no martial arts and what there was appe...,almost martial art appeared simulated camera f...,almost no martial arts and what there was appe...,almost martial art appeared simulated camera f...
287681,"rt @t3hd1g17: @mihhamd это прозвучало как ""вы ...",2,RuTweetCorp,tweet,ru,"rt @t3hd1g17: @mihhamd it sounded like ""do you...",0,rt t3hd1g17 mihhamd it sounded like do you wan...,rt t3hd1g17 mihhamd sounded like want believe ...,rt @t3hd1g17 @mihhamd it sounded like do you w...,rt @t3hd1g17 @mihhamd sounded like want believ...
...,...,...,...,...,...,...,...,...,...,...,...
492892,сначала о плюсах приветливый персонал вкусная ...,1,clothing e-comm,review,ru,"first, about the pros: friendly staff, delicio...",0,first about the pros friendly staff delicious ...,first pro friendly staff delicious cuisine pro...,first about the pros friendly staff delicious ...,first pro friendly staff delicious cuisine pro...
403600,"и социопатите, които контролират тези програми...",1,XED,sentence,bg,and the sociopaths who control these programs ...,0,and the sociopaths who control these programs ...,sociopath control program must ask stop talkin...,and the sociopaths who control these programs ...,sociopath control program must ask stop talkin...
291898,"rt @yarishka_cao: ебааать, на майдане стоят ми...",0,RuTweetCorp,tweet,ru,"rt @yarishka_cao: fuck, there are millions on ...",0,rt yarishka cao fuck there are millions on the...,rt yarishka cao fuck million maidan russia say...,rt @yarishka_cao fuck there are millions on th...,rt @yarishka_cao fuck million maidan russia sa...
117782,adică cel pe care l-a lăsat înăuntru a deschis...,0,XED,sentence,ro,so whoever he let in opened the windows to mak...,0,so whoever he let in opened the windows to mak...,whoever let opened window make look like burglary,so whoever he let in opened the windows to mak...,whoever let opened window make look like burglary


In [12]:
class Collator(object):
    def __init__(self, true_max_len, x_column, y_column):
        self.true_max_len = true_max_len
    def __call__(self, batch):
        max_len = max(len(row[x_column]) for row in batch)
        input_embeds = []
        labels = torch.empty(len(batch), dtype=torch.long)
        for idx, row in enumerate(batch):
            input_embeds.append(row[x_column])
            labels[idx] = row[y_column]
        input_embeds[0] = nn.ConstantPad1d((0, self.true_max_len - input_embeds[0].shape[0]), 0)(input_embeds[0])
        input_embeds = pad_sequence(input_embeds,batch_first=True)
        input_embeds = torch.Tensor(input_embeds)
        return {x_column: input_embeds, y_column: labels}  

In [15]:
def create_embeddings(df, text_column, label_column, model, 
                      emb_type, min_length, max_length, train_test_rate, 
                      datatype_column, test_type):
    if datatype_column is not None:
        df = df[[text_column, label_column, datatype_column]]
    else:
        df = df[[text_column, label_column]]
    df['token'] = [[]]*df.shape[0]
    if emb_type == 'mean':
        for i in trange(df.shape[0], leave=False, desc='Applying emb'):
            temp = list(df[text_column][i].split())[:max_length]
            temp2 = make_emb(model, temp, emb_type, min_length, max_length)
            df['token'][i] = temp2
            temp = None; del temp
    elif emb_type == 'sequence':
        for i in trange(df.shape[0], leave=False, desc='Applying emb'):
            temp = list(df[text_column][i].split())[:max_length]
            temp2 = [encode(word) for word in temp]
            if len(temp2) < min_length or len(temp2) > max_length:
                temp2 = []
            df['token'][i] = temp2
            temp = None; del temp
    df = df.loc[df['token'].isin([[]])==False]
    df.reset_index(drop=True, inplace=True)
    if train_test_rate > 0.:
        if test_type == 'all':
            train_df, test_df = train_test_split(df, test_size=train_test_rate, random_state=42,stratify=df[label_column])
        else:
            train_df = df[df['dataset'] != test_type]
            test_df = df[df['dataset'] == test_type]         
        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)      
        return train_df, test_df
    else:
        return df


In [16]:
max_length = 128

# collate_fn = Collator(max_length,x_column=x_column, y_column=y_column)
# train_df, test_df = create_embeddings(df=df1, text_column='lemma', label_column='label',
#                             model=word2vec, emb_type='sequence', min_length=3, 
#                             max_length=max_length,  train_test_rate = 0.25,
#                             datatype_column='dataset',test_type='all', balancing='no',
#                             batch_size=512, collate_fn=collate_fn)
df_all = create_embeddings(df=df1, text_column='lemma', label_column='label',
                            model=word2vec, emb_type='sequence', min_length=3, 
                            max_length=max_length, train_test_rate = 0.,
                            datatype_column='dataset', test_type='all')

Applying emb:   0%|          | 0/584766 [00:00<?, ?it/s]

In [17]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)             
    elif isinstance(m, nn.LSTM) or isinstance(m, nn.GRU) or isinstance(m, nn.RNN):
        for name, param in m.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0)
            elif 'weight' in name:
                nn.init.xavier_uniform(param)

In [18]:
class GRUModel(nn.Module):
    def __init__(self, use_embed_layer, embed_size, hidden_size, drop_rate, sequence_length, layer_type, bidir_flg, num_layers=6, num_classes=3):
        super().__init__()
        self.hidden_size  = hidden_size
        self.num_layers = num_layers        
        self.layer_type = layer_type
        self.bidir = True if bidir_flg == 1 else False
        self.drop_in_gru = drop_rate
      
        if use_embed_layer:
            self.embed = nn.Embedding(len(word2idx), embed_size)
        else:
            self.embed = nn.Linear(len(word2idx), embed_size)
        if self.layer_type == 'gru':
            self.rnn = nn.GRU(embed_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=self.bidir,dropout=self.drop_in_gru)
        elif self.layer_type == 'rnn':
            self.rnn = nn.RNN(embed_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=self.bidir,dropout=self.drop_in_gru)
        if self.layer_type == 'lstm':
            self.rnn = nn.LSTM(embed_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=self.bidir,dropout=self.drop_in_gru)
        self.fc = nn.Linear(hidden_size*sequence_length*2 if self.bidir else hidden_size*sequence_length, num_classes)
        self.drop = nn.Dropout(drop_rate)

    def init_hidden(self, batch_size, layer_type):
        if layer_type == 'lstm':
            return(autograd.Variable(torch.randn(self.num_layers*2 if self.bidir else self.num_layers, batch_size, self.hidden_size)).to(device),
               autograd.Variable(torch.randn(self.num_layers*2 if self.bidir else self.num_layers, batch_size, self.hidden_size)).to(device))
        elif layer_type == 'gru':
            return(autograd.Variable(torch.randn(self.num_layers*2 if self.bidir else self.num_layers, batch_size, self.hidden_size)).to(device))

    
    def forward(self, x):
        x = self.embed(x)
        if self.layer_type == 'lstm':
            self.hidden = self.init_hidden(x.size(0),self.layer_type)
            x, (_,_) = self.rnn(x, self.hidden)
        elif self.layer_type == 'rnn':
            x, _ = self.rnn(x)
        else:
            self.hidden = self.init_hidden(x.size(0),self.layer_type)
            x, _ = self.rnn(x, self.hidden)
        x = F.relu(x)        
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [19]:
def freeze_embeddings(model, req_grad=False):
    embeddings = model.embed
    for c_p in embeddings.parameters():
        c_p.requires_grad = req_grad
        
def training(model, x_column, y_column, criterion, optimizer, scheduler, num_epochs, loaders,
                    row, df_result, max_grad_norm=2, num_freeze_iter=1000):
    freeze_embeddings(model)
    for e in trange(num_epochs, leave=False, desc='Epoch'):
        time_start = time.time()
        model.train()
        num_iter = 0
        pbar = tqdm(loaders["train"], leave=False, desc='Train')
        y_true_train = []
        y_pred_train = []
        num_iter = 0
        train_loss = 0
        num_errors = 0
#         for batch_idx, batch in enumerate(loaders["train"]):
        for batch in pbar:
#             print(batch)
            if num_iter > num_freeze_iter and e < 1:
                freeze_embeddings(model, True)
            optimizer.zero_grad()
            input_embeds = batch[x_column].to(device)
            labels = batch[y_column].to(device)
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            train_loss += loss.item()
            for idx, i in enumerate(prediction):
                y_pred_train.append(torch.argmax(i).item())
            for item in labels:
                y_true_train.append(item.item())
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
            input_embeds.to(torch.device("cpu"))
            labels.to(torch.device("cpu"))
            prediction.to(torch.device("cpu"))
            loss.to(torch.device("cpu"))         
            input_embeds, labels, prediction, loss = None, None, None, None
            del input_embeds, labels, prediction, loss
        if scheduler is not None:
            scheduler.step()    
        train_loss = train_loss / (num_iter+1)
        train_acc = round(accuracy_score(y_true_train, y_pred_train),3)
        train_f1 = round(f1_score(y_true_train, y_pred_train, average='weighted'),3)
        model.eval()
        with torch.no_grad():
            num_iter = 0
            y_pred_test = []
            y_true_test = []
            test_loss = 0
            num_errors = 0
            pbar_test_eval = tqdm(loaders["test"], leave=False, desc='Test')
            for batch in pbar_test_eval:
                input_embeds = batch[x_column].to(device)
                labels = batch[y_column].to(device)
                prediction = model(input_embeds)
                loss = criterion(prediction, labels)
                test_loss += loss.item()
                for idx, i in enumerate(prediction):
                    y_pred_test.append(torch.argmax(i).item())
                for item in labels:
                    y_true_test.append(item.item())
                num_iter += 1
                input_embeds.to(torch.device("cpu"))
                labels.to(torch.device("cpu"))
                prediction.to(torch.device("cpu"))
                loss.to(torch.device("cpu"))
                input_embeds, labels, prediction, loss = None, None, None, None
                del input_embeds, labels, prediction, loss
               

        test_loss = test_loss / (num_iter+1)
        test_acc = round(accuracy_score(y_true_test, y_pred_test),3)
        test_f1 = round(f1_score(y_true_test, y_pred_test, average='weighted'),3)
        time_stop = time.time()
        print(f"epoch: {e}, train loss: {round(train_loss,4)}, test Loss: {round(test_loss,4)}, train acc: {train_acc}, test acc: {test_acc}, time:{round(time_stop-time_start)}")
        ans = [round(train_loss,4), train_acc, train_f1, round(test_loss,4), test_acc, test_f1]
        temp_row = row
        temp_row.append(e)
        for item in ans:
            temp_row.append(item)
        temp_row.append(round(time_stop-time_start))
        df_result.loc[df_result.shape[0]] = temp_row
        for i in range(len(ans)+2):
            temp_row.pop(-1)
     
        with torch.no_grad():
            torch.cuda.empty_cache()
        y_true_train, y_pred_train, y_pred_test, y_true_test, temp_row = None, None, None, None, None
        del y_true_train, y_pred_train, y_pred_test, y_true_test, temp_row
        gc.collect()
    return df_result
    

In [20]:
def create_dataloaders_kfold(df, n_split, batch_size, collate_fn, get_one, num_iter):
    loaders = {}
    df = shuffle(df)
    df.reset_index(drop=True, inplace=True)
    if get_one:
        temp_train_df = pd.concat([df_all.loc[df_all.index<df_all.shape[0]/5*num_iter], df_all.loc[df_all.index>=df_all.shape[0]/5*(num_iter+1)]])
        temp_test_df = pd.concat([df_all.loc[df_all.index>=df_all.shape[0]/5*num_iter], df_all.loc[df_all.index<df_all.shape[0]/5*(num_iter+1)]])
        temp_train_df.reset_index(drop=True, inplace=True)
        temp_test_df.reset_index(drop=True, inplace=True)
        dataset = DatasetDict()
        dataset['train'] = Dataset.from_pandas(temp_train_df)
        dataset['test'] = Dataset.from_pandas(temp_test_df)
        dataset.set_format(type='torch')
        loaders = {'train': torch.utils.data.DataLoader(
                      dataset['train'], collate_fn=collate_fn,
                      batch_size=batch_size,shuffle=True),
                     'test': torch.utils.data.DataLoader(
                      dataset['test'], collate_fn=collate_fn,
                      batch_size=batch_size,shuffle=False)}
        temp_train_df,temp_test_df = None, None; del temp_train_df,temp_test_df; gc.collect()
        return loaders
        
    for i in range(n_split):
        temp_train_df = pd.concat([df_all.loc[df_all.index<df_all.shape[0]/5*num_iter], df_all.loc[df_all.index>=df_all.shape[0]/5*(num_iter+1)]])
        temp_test_df = pd.concat([df_all.loc[df_all.index>=df_all.shape[0]/5*num_iter], df_all.loc[df_all.index<df_all.shape[0]/5*(num_iter+1)]])
        temp_train_df.reset_index(drop=True, inplace=True)
        temp_test_df.reset_index(drop=True, inplace=True)
        dataset = DatasetDict()
        dataset['train'] = Dataset.from_pandas(temp_train_df)
        dataset['test'] = Dataset.from_pandas(temp_test_df)
        dataset.set_format(type='torch')
        loaders[i] = {'train': torch.utils.data.DataLoader(
                      dataset['train'], collate_fn=collate_fn,
                      batch_size=batch_size,shuffle=True),
                     'test': torch.utils.data.DataLoader(
                      dataset['test'], collate_fn=collate_fn,
                      batch_size=batch_size,shuffle=False)}
        temp_train_df,temp_test_df = None, None; del temp_train_df,temp_test_df; gc.collect()
    return loaders

In [26]:
use_embed_layer = True
hidden_size = 200
drop_rate = 0.1
max_length = 128
layer_type = 'lstm'
bidir_flg = 0
lr = 0.001
num_epochs=1
num_layers=2
max_grad_norm = 1.0
x_column = 'token'
y_column = 'label'
batch_size = 512
n_split = 3
collate_fn = Collator(max_length,x_column=x_column, y_column=y_column)
columns=['layer_type','num_layer','bidir_flg','hidden_size','drop_rate','batch_size','num_iter',
         'epoch','train_loss','train_acc','train_f1',
         'test_loss','test_acc','test_f1','time']
df_result = pd.DataFrame([], columns=columns)

for num_iter in range(n_split):
    row = [layer_type, num_layers, bidir_flg, hidden_size, drop_rate, batch_size, num_iter]
    model = GRUModel(use_embed_layer, word2vec.vector_size, hidden_size, 
                 drop_rate,max_length,layer_type, bidir_flg,num_layers).to(device)
    model.apply(init_weights)
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=3, gamma=0.9)
    loaders = create_dataloaders_kfold(df=df_all, n_split=n_split, batch_size=batch_size, collate_fn=collate_fn, get_one=True, num_iter=num_iter)
    df_result = training(model, x_column, y_column, criterion, optimizer, scheduler, num_epochs, loaders, row, df_result, max_grad_norm) 
df_result

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Train:   0%|          | 0/789 [00:00<?, ?it/s]

Test:   0%|          | 0/1183 [00:00<?, ?it/s]

epoch: 0, train loss: 1.075, test Loss: 1.0563, train acc: 0.383, test acc: 0.405, time:202


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Train:   0%|          | 0/789 [00:00<?, ?it/s]

Test:   0%|          | 0/1183 [00:00<?, ?it/s]

epoch: 0, train loss: 1.0479, test Loss: 1.0696, train acc: 0.407, test acc: 0.39, time:195


,layer_type,num_layer,bidir_flg,hidden_size,drop_rate,batch_size,num_iter,epoch,train_loss,train_acc,train_f1,test_loss,test_acc,test_f1,time
0,lstm,2,0,200,0.1,512,0,0,1.0750,0.383,0.333,1.0563,0.405,0.360,202
1,lstm,2,0,200,0.1,512,1,0,1.0311,0.418,0.364,1.0968,0.379,0.297,197
2,lstm,2,0,200,0.1,512,2,0,1.0479,0.407,0.363,1.0696,0.390,0.337,195
